# Merge two or more geojsons files
- [geopandas.org](https://geopandas.org/en/stable/index.html)
- [folium](https://python-visualization.github.io/folium/index.html)

In [1]:
from pathlib import Path

import folium
import geopandas
import pandas as pd
from branca.element import Figure

In [2]:
DATA_PATH = Path().resolve().parent.parent.parent/"data"
print(DATA_PATH)

/home/pyuser/workspace/data


## Load data

In [3]:
gdf_pt_continente_distritos = geopandas.read_file(
        DATA_PATH/"datasets"/"geojson"/"continente_distritos.geojson"
)
gdf_pt_continente_distritos.head()

,DI,Distrito,N_Freguesi,N_Concelho,Area_Ha,geometry
0,01,AVEIRO,147,19,280094.16,"MULTIPOLYGON (((-8.27202 41.06475, -8.26862 41..."
1,02,BEJA,75,14,1026332.49,"MULTIPOLYGON (((-7.28303 38.32993, -7.28298 38..."
2,03,BRAGA,347,14,270610.54,"MULTIPOLYGON (((-8.04964 41.81636, -8.04910 41..."
3,04,BRAGANÇA,226,12,659853.71,"MULTIPOLYGON (((-6.79998 41.98916, -6.79790 41..."
4,05,CASTELO BRANCO,120,11,662747.31,"MULTIPOLYGON (((-7.29277 40.41613, -7.29026 40..."


In [4]:
gdf_pt_madeira_ilhas = geopandas.read_file(
        DATA_PATH/"datasets"/"geojson"/"madeira_ilhas.geojson"
)
gdf_pt_madeira_ilhas.head()

,ID1,Codigo,Ilha,DI,N_Freguesi,Area_Ha,N_Municipi,geometry
0,487,02010105,Ilha de Porto Santo (Madeira),32,1,4300.80000,1,"MULTIPOLYGON (((-16.28613 33.12429, -16.28620 ..."
1,491,02010105,Ilha da Madeira (Madeira),31,53,75850.90000,10,"MULTIPOLYGON (((-17.16369 32.86912, -17.16372 ..."


In [5]:
gdf_pt_acores_ilhas = geopandas.read_file(
        DATA_PATH/"datasets"/"geojson"/"acores_ilhas.geojson"
)
gdf_pt_acores_ilhas.head()

,NUT1_COD,NUT1_DSG,N_FREGUESI,AREA_HA,geometry
0,2,REGIÃO AUTÓNOMA DOS AÇORES,156,232196.24,"MULTIPOLYGON (((-28.47257 38.56002, -28.47230 ..."
1,2,REGIÃO AUTÓNOMA DOS AÇORES,156,232196.24,"MULTIPOLYGON (((-31.20810 39.52441, -31.20795 ..."
2,2,REGIÃO AUTÓNOMA DOS AÇORES,156,232196.24,"MULTIPOLYGON (((-25.08858 37.01656, -25.08853 ..."


In [6]:
gdf_es_communities = geopandas.read_file(
        DATA_PATH/"datasets"/"geojson"/"spain_communities.geojson"
)
gdf_es_communities.head()

,region,poblacion_,cartodb_id,name,geometry
0,01,8424102,161,Andalusia,"MULTIPOLYGON (((-3.03454 35.93810, -3.03539 35..."
1,02,1346293,116,Aragon,"POLYGON ((0.22035 41.07143, 0.21970 41.07074, ..."
2,03,1113114,7,Ilhas Baleares,"MULTIPOLYGON (((1.47914 38.71071, 1.47907 38.7..."
3,04,2126769,73,Ilhas Canarias,"MULTIPOLYGON (((-18.16100 27.71877, -18.16105 ..."
4,05,593121,39,Cantabria,"MULTIPOLYGON (((-3.26022 43.26358, -3.25673 43..."


## Merge geojsons

In [7]:
merged_regions = (
    pd.concat([
        gdf_pt_continente_distritos.filter(["Distrito", "geometry"]).rename(columns={"Distrito": "location"}).to_crs("WGS 84"),
        gdf_pt_madeira_ilhas.filter(["Ilha", "geometry"]).rename(columns={"Ilha": "location"}).to_crs("WGS 84"),
        gdf_pt_acores_ilhas.filter(["NUT1_DSG", "geometry"]).rename(columns={"NUT1_DSG": "location"}).to_crs("WGS 84"),
        gdf_es_communities.filter(["name", "geometry"]).rename(columns={"name": "location"}).to_crs("WGS 84")
    ])
    .assign(location=lambda x: x["location"].str.normalize("NFKC").str.title())
)
merged_regions.head()

,location,geometry
0,Aveiro,"MULTIPOLYGON (((-8.27202 41.06475, -8.26862 41..."
1,Beja,"MULTIPOLYGON (((-7.28303 38.32993, -7.28298 38..."
2,Braga,"MULTIPOLYGON (((-8.04964 41.81636, -8.04910 41..."
3,Bragança,"MULTIPOLYGON (((-6.79998 41.98916, -6.79790 41..."
4,Castelo Branco,"MULTIPOLYGON (((-7.29277 40.41613, -7.29026 40..."


## Export data

## Extra: show on map using Folium